# Step0. 라이브러리 설치, 불러오기 및 버전 확인

In [ ]:
!pip install pmdarima

!pip install 'statsmodels==0.12.2' --force-reinstall

In [ ]:
import os
import requests
from io import BytesIO
from itertools import product
from datetime import datetime

# 데이터 처리
import pandas as pd
import numpy as np

# 시각화
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns

# 분석
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
import statsmodels.api as sm
import pmdarima as pm  

# statsmodels
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller # 정상성 검정
from statsmodels.tsa.arima_model import ARIMA

import warnings
warnings.filterwarnings( 'ignore' )

print("c")

In [ ]:
print( f'# pandas:       {pd.__version__}' ) # >=0.19
print( f'# statsmodels:  {sm.__version__}' ) # >=0.11
print( f'# numpy:        {np.__version__}' ) # >=1.17.3
print( f'# scikit-learn: {sklearn.__version__}' ) # >=0.22
print( f'# pmdarima:     {pm.__version__}' )

In [ ]:
file_path_raws = '/content/drive/MyDrive/Aiffel/Going_deeper/demand_forecasting/raws.csv'
df_raws = pd.read_csv(file_path_raws)

df_op_ts = df_raws.set_index('target_dates').sort_index()
df_op_ts.head()

# Step1. 학습 데이터와 테스트 데이터 나누기

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['click_d_1', 'click_d_2', 'click_d_3', 'click_d_4', 'click_d_5', 'click_d_6', 'click_d_7', 'is_clean', 'avg_precipitation', 'avg_temperature', 'target_dates']]

_x_weekday = pd.get_dummies(df_raws['weekday'], prefix='weekday', drop_first=False)

_x_data = pd.concat([_x_data, _x_weekday], axis=1)

_x_data.set_index("target_dates", drop=True, inplace=True)
_x_data = _x_data.sort_values('target_dates')

# 예측 대상 데이터 설정
target = df_raws[['op_rate_0d_all_cars', 'target_dates' ]]

target.set_index("target_dates", drop=True, inplace=True)
target = target.sort_values('target_dates')

# Train/Test 데이터셋 나누기
_x_data_train = _x_data[_x_data.index<'2021-04-01']
_x_data_test = _x_data[_x_data.index>='2021-04-01']
target_train = target[target.index<'2021-04-01']
target_test = target[target.index>='2021-04-01']

In [ ]:
x_train = _x_data_train
x_test = _x_data_test
y_train = target_train
y_test = target_test

# Step2. Grid search를 이용하여 최상의 파라미터 조합 찾기

In [ ]:
# itertools의 product함수를 사용해 가능한 모든 파라미터 조합 생성
import itertools

## Define the p, d and q parameters to take any value between 0 and 1
p = d = q = range(0, 2)

## Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))

# generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]

In [ ]:
# for 문으로 최소 AIC 찾기 (Grid Search)
best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None
tmp_model = None
best_mdl = None
iter_cnt = 1

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            tmp_mdl = sm.tsa.statespace.SARIMAX(y_train, exog=x_train, order = param,
                                                seasonal_order = param_seasonal,
                                                enforce_stationarity=True,
                                                enforce_invertibility=True)
            
            res = tmp_mdl.fit() 
            print("Iter{}/64: SARIMAX{}x{}, AIC:{}".format(iter_cnt, param, param_seasonal, res.aic))
            iter_cnt+=1
            if res.aic < best_aic:
                best_aic = res.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
                best_mdl = tmp_mdl
        except:
            #print("Unexpected error:", sys.exc_info()[0])
            continue
print("Best SARIMAX{}x{} model - AIC:{}".format(best_pdq, best_seasonal_pdq, best_aic))

찾은 최상의 파라미터 조합은 (1,0,1)(1,0,1,7)이다.

# Step3. 모형 구조 확인

In [ ]:
## SARIMAX모델 정의
model = sm.tsa.statespace.SARIMAX(endog=y_train, order=(1, 0, 1),
                                seasonal_order=(1, 0, 1, 7),
                                enforce_stationarity=True,
                                enforce_invertibility=True) # enforce_stationarity는 AR항이, _inveribility는 MA항이 stationary를 만족하게끔 강제하는 option

res = model.fit(disp=-1)

In [ ]:
res.summary()

# Step4. 예측

In [ ]:
#RMSE
pred = res.get_forecast(steps=52, exog=x_test)
pred_ci = pred.conf_int()

y_forcasted = pred.predicted_mean

rmse = np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_forcasted))
print(f'Test RMSE: {rmse}\n')

In [ ]:
#시각화
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
plt.xticks(rotation=45)
loc = plticker.MultipleLocator(base=7.0) 
ax.xaxis.set_major_locator(loc)

plt.plot(y_train[-50:], alpha=0.5, color='black', label='training')
plt.plot(y_test, alpha=0.5, color='red', label='actual operation rate')
plt.plot(target_test.index.values, y_forcasted, alpha=0.5, color='blue', label='predicted operation rate')
plt.fill_between(target_test.index.values, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], alpha=0.1, color='b')

plt.legend()
plt.show()

#회고

이번 프로젝트 노드를 진행하던 당일, 외주 작업 클라이언트로부터 수정 사항 뭉텅이를 통보받은 나는 야간 게더에서 다른 분들이 프로젝트 수행하시는 걸 구경하며 영상 편집을 했다(외주 작업 할거면서 야간게더는 왜 들어갔냐?  지호님 빈님과 무슨 일이 있어도 야간게더에서 만나기로 도원결의를 했기 때문이다).
당시 지호님, 빈님, 지연님, 승민님이 고통받으며 코드를 수정하고 또 수정하는 모습을 보고 발등에 불이 떨어진 수준을 넘어, 발등이 활활 타고 있음을 느꼈지만 그렇다고 위약금을 물 수는 없으므로 실시간으로 눈물과 겁을 집어 삼키며 영상을 편집했더랬다.   
수정사항을 얼추 쳐내고 드디어 똑바로 마주한 going deeper의 첫 번째 프로젝트. 두려운 마음으로 프로젝트에 필요한 코드를 본문에서 옮기고, 파라미터를 바꾸고, 필요한 부분은 구글링 하여 작성해 넣고(더하여 파이토치 풀잎시간에 정연님, 선아님, 성진님과 이 프로젝트 관련해 짤막하게 이야기를 나누다가 중간에 프로젝트에서 요구한 인자 값(exog=x_train)을 넣지 않은 걸 발견했다. 정연님 감사합니다!!)… 짠! 너무나도 쉽게 루브릭 1번 기준을 달성했다. 아무 오류도 뱉지 않은 코드를 어리둥절하게 쳐다보며, 이게 맞나 의심했지만 지호님, 빈님, 지연님께 여쭤보니 이게 맞다고 한다.   
다른 분들의 시행착오를 곁눈질해 “저 부분에서 저런 오류가 나기도 하는구나”, 미리 파악한 것이 크게 작용했다. 올무가 어디 있는지 알고 뛰는 노루였다고나 할까? 덕분에 발목이 무사한채로 산 정상까지 완주한 것이다.   
루브릭 2번, 3번이 남았지만 나에겐 아직 수정할 영상도 남았으므로 이대로 제출한다. 별 하나에 사랑과, 별 하나에 추억과, 별 하나에 기타 등등(단, 어머니 빼고)…을 담아서~